In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from statsmodels.tsa.stattools import adfuller
import seaborn as sns

In [2]:
raw_df = pd.read_csv('data/zillow_data.csv')

Filter only for Texas
<br>Drop Metro because there were many nulls, and it was redundant with other location data 
<br> Drop regionID and SizeRank because values were specific to Zillow 

In [3]:
raw_df = raw_df[raw_df['State']=='TX']
raw_df.drop(raw_df.columns[np.r_[0,3,4, 6:100]], axis=1, inplace=True)
raw_df = raw_df.rename(columns={'RegionName': 'zipcode'})


In [4]:
#Calculate historical return on investment
raw_df['ROI']= (raw_df['2018-04']/raw_df['2004-01'])-1
raw_df['ROI']


#Calculate standard deviation of monthly values
raw_df['std']=raw_df.loc[:,'2004-01':'2018-04'].std(skipna=True, axis=1)

#Calculate historical mean value
raw_df['mean']=raw_df.loc[:,'2004-01':'2018-04'].mean(skipna=True, axis=1)

#Calculate coefficient of variance
raw_df['CV']=raw_df['std']/raw_df['mean']

#Show calculated values
raw_df[['zipcode','std','mean','ROI','CV']].head()



,zipcode,std,mean,ROI,CV
1,75070,38257.775512,227040.116279,0.714438,0.168507
2,77494,33208.367605,266633.139535,0.423210,0.124547
4,79936,10248.462666,113729.651163,0.446429,0.090112
5,77084,14170.210695,130143.604651,0.376047,0.108881
8,77449,16508.811448,134945.348837,0.422115,0.122337


In [5]:
raw_df

,zipcode,City,CountyName,2004-01,2004-02,2004-03,2004-04,2004-05,2004-06,2004-07,...,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,ROI,std,mean,CV
1,75070,McKinney,Collin,187700.0,188800.0,190300.0,191800.0,193000.0,193900.0,194500.0,...,315000,316600,318100,319600,321100,321800,0.714438,38257.775512,227040.116279,0.168507
2,77494,Katy,Harris,231800.0,233100.0,233500.0,233000.0,232100.0,231300.0,230700.0,...,320800,321200,321200,323000,326900,329900,0.423210,33208.367605,266633.139535,0.124547
4,79936,El Paso,El Paso,84000.0,84700.0,85500.0,86400.0,87200.0,88000.0,88900.0,...,120300,120300,120300,120500,121000,121500,0.446429,10248.462666,113729.651163,0.090112
5,77084,Houston,Harris,119400.0,120400.0,121200.0,121900.0,122400.0,122700.0,123000.0,...,162800,162800,162800,162900,163500,164300,0.376047,14170.210695,130143.604651,0.108881
8,77449,Katy,Harris,123900.0,125300.0,126600.0,127500.0,128100.0,128500.0,128800.0,...,170900,172300,173300,174200,175400,176200,0.422115,16508.811448,134945.348837,0.122337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14372,76941,Mertzon,Irion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,123500,124700,124300,122600,121600,121600,NaN,13734.029878,99318.367347,0.138283
14472,79313,Anton,Hockley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,58900,61500,63000,63600,63500,63300,NaN,5286.674724,54424.675325,0.097137
14492,79355,Plains,Yoakum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,99700,97700,95800,94600,94000,93500,NaN,12929.465872,78967.346939,0.163732
14599,79366,Ransom Canyon,Lubbock,161900.0,162300.0,162700.0,163100.0,163400.0,163600.0,163900.0,...,251300,251500,251700,252500,255000,257500,0.590488,25909.956515,189098.837209,0.137018


In [6]:
def melt_data(raw_df):
    melted = pd.melt(raw_df, id_vars=['zipcode', 'City', 'CountyName','ROI','std','mean','CV'], var_name='date')
    melted['date'] = pd.to_datetime(melted['date'], infer_datetime_format=True)
    melted['year'] = [d.year for d in melted.date]
    melted = melted.dropna(subset=['value'])
    return melted

In [7]:
df = melt_data(raw_df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167292 entries, 0 to 170107
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   zipcode     167292 non-null  int64         
 1   City        167292 non-null  object        
 2   CountyName  167292 non-null  object        
 3   ROI         162884 non-null  float64       
 4   std         167292 non-null  float64       
 5   mean        167292 non-null  float64       
 6   CV          167292 non-null  float64       
 7   date        167292 non-null  datetime64[ns]
 8   value       167292 non-null  float64       
 9   year        167292 non-null  int64         
dtypes: datetime64[ns](1), float64(5), int64(2), object(2)
memory usage: 14.0+ MB


In [8]:
df.set_index(df['date'], inplace = True)
df.drop('date',axis=1, inplace=True)
df

,zipcode,City,CountyName,ROI,std,mean,CV,value,year
date,,,,,,,,,
2004-01-01,75070,McKinney,Collin,0.714438,38257.775512,227040.116279,0.168507,187700.0,2004
2004-01-01,77494,Katy,Harris,0.423210,33208.367605,266633.139535,0.124547,231800.0,2004
2004-01-01,79936,El Paso,El Paso,0.446429,10248.462666,113729.651163,0.090112,84000.0,2004
2004-01-01,77084,Houston,Harris,0.376047,14170.210695,130143.604651,0.108881,119400.0,2004
2004-01-01,77449,Katy,Harris,0.422115,16508.811448,134945.348837,0.122337,123900.0,2004
...,...,...,...,...,...,...,...,...,...
2018-04-01,76941,Mertzon,Irion,NaN,13734.029878,99318.367347,0.138283,121600.0,2018
2018-04-01,79313,Anton,Hockley,NaN,5286.674724,54424.675325,0.097137,63300.0,2018
2018-04-01,79355,Plains,Yoakum,NaN,12929.465872,78967.346939,0.163732,93500.0,2018


In [9]:
df_income = pd.read_csv('data/zip_codes_2019_median_inc.csv')
df_income['state'] = df_income['Preferred name'].str.slice(-2)
df_income = df_income[df_income['state']=='TX']
df_income = (df_income[df_income['Type'] != 'PO box'])
df_income.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1839 entries, 31142 to 34261
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ZIP Code                     1839 non-null   int64  
 1   Type                         1839 non-null   object 
 2   State FIPS                   1839 non-null   int64  
 3   Preferred name               1839 non-null   object 
 4   Alternate names              829 non-null    object 
 5   Population (2019)            1718 non-null   object 
 6   Housing units (2019)         1718 non-null   object 
 7   Median family income (2019)  1623 non-null   object 
 8   MFI percentile (2019)        1623 non-null   float64
 9   Latitude                     1718 non-null   float64
 10  Longitude                    1718 non-null   float64
 11  Land area                    1718 non-null   float64
 12  Water area                   1718 non-null   float64
 13  state        

In [10]:
df_income.drop(df_income.columns[np.r_[1:5,6,8,11,12]], axis=1, inplace=True)
df_income.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1839 entries, 31142 to 34261
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ZIP Code                     1839 non-null   int64  
 1   Population (2019)            1718 non-null   object 
 2   Median family income (2019)  1623 non-null   object 
 3   Latitude                     1718 non-null   float64
 4   Longitude                    1718 non-null   float64
 5   state                        1839 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 100.6+ KB


Clean up column names

In [11]:
df_income = df_income.rename(columns={'Median family income (2019)': 'median_family_income',
                               'Population (2019)': 'population',
                               'ZIP Code': 'zipcode'})

In [12]:
df_income['median_family_income']

31142         NaN
31143         NaN
31744     $79,551
31745    $105,656
31746     $72,325
           ...   
34257         NaN
34258         NaN
34259         NaN
34260         NaN
34261         NaN
Name: median_family_income, Length: 1839, dtype: object

In [13]:
df_income['median_family_income'] = df_income['median_family_income'].str.strip('$\n\t')
df_income['median_family_income'] = df_income['median_family_income'].str.replace(',','')
df_income  = df_income[df_income['median_family_income'].notna()]
df_income['median_family_income'] = df_income['median_family_income'].astype(int)



 

In [14]:
df_income

,zipcode,population,median_family_income,Latitude,Longitude,state
31744,75001,"14,992",79551,32.959999,-96.838997,TX
31745,75002,"71,253",105656,33.090000,-96.609001,TX
31746,75006,"51,642",72325,32.962002,-96.899002,TX
31747,75007,"55,500",96094,33.005001,-96.897003,TX
31748,75009,"14,089",117857,33.339001,-96.752998,TX
...,...,...,...,...,...,...
34233,79932,"28,243",75676,31.875000,-106.608002,TX
34234,79934,"26,445",71037,31.951000,-106.433998,TX
34235,79935,"17,850",61607,31.768000,-106.329002,TX
34236,79936,"111,620",61016,31.777000,-106.296997,TX


In [15]:
df_income['population'] = df_income['population'].str.replace(',','')
df_income = df_income[df_income['population'].notna()]
df_income['population'] = df_income['population'].astype(int)

In [16]:
df_income.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1623 entries, 31744 to 34238
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   zipcode               1623 non-null   int64  
 1   population            1623 non-null   int64  
 2   median_family_income  1623 non-null   int64  
 3   Latitude              1623 non-null   float64
 4   Longitude             1623 non-null   float64
 5   state                 1623 non-null   object 
dtypes: float64(2), int64(3), object(1)
memory usage: 88.8+ KB


In [17]:
df_income.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1623 entries, 31744 to 34238
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   zipcode               1623 non-null   int64  
 1   population            1623 non-null   int64  
 2   median_family_income  1623 non-null   int64  
 3   Latitude              1623 non-null   float64
 4   Longitude             1623 non-null   float64
 5   state                 1623 non-null   object 
dtypes: float64(2), int64(3), object(1)
memory usage: 88.8+ KB


In [18]:
df_merged = df.reset_index().merge(df_income, how="left").set_index('date')


In [19]:
df_merged.describe().T

,count,mean,std,min,25%,50%,75%,max
zipcode,167292.0,77169.453471,1364.146918,75001.000000,76018.000000,77098.000000,78253.000000,7.993800e+04
ROI,162884.0,0.555032,0.220363,-0.301587,0.433762,0.568088,0.690932,1.492424e+00
std,167292.0,19219.935086,14698.179659,1461.036168,9647.626333,15206.650804,24187.507432,1.144473e+05
mean,167292.0,151867.811969,101194.828605,32718.292683,88525.581395,125155.813953,179225.000000,9.569622e+05
CV,167292.0,0.121636,0.036192,0.019827,0.098565,0.120967,0.142339,2.930644e-01
value,167292.0,151867.811969,104030.853299,26000.000000,87100.000000,125000.000000,177300.000000,1.268600e+06
year,167292.0,2010.736628,4.136874,2004.000000,2007.000000,2011.000000,2014.000000,2.018000e+03
population,165400.0,25741.887219,20378.618340,1056.000000,9565.000000,21644.000000,37090.000000,1.282940e+05
median_family_income,165400.0,77547.462310,32562.627305,21169.000000,56013.000000,69628.000000,90707.000000,2.500010e+05
Latitude,165400.0,31.123359,1.806161,25.944000,29.736000,30.884001,32.743000,3.604600e+01


# Filtering

Get IQR of data based on salaries  

In [20]:
# Select the first quantile
q1 = df_merged['value'].quantile(.25)

# Select the third quantile
q3 = df_merged['value'].quantile(.75)


# Create a filtered in between q1 & q3
filtered =df_merged[(df_merged['value']>=q1) 
             & (df_merged['value']<=q3)]
df_merged = pd.DataFrame(filtered)
df_merged

,zipcode,City,CountyName,ROI,std,mean,CV,value,year,population,median_family_income,Latitude,Longitude,state
date,,,,,,,,,,,,,,
2004-01-01,77084,Houston,Harris,0.376047,14170.210695,130143.604651,0.108881,119400.0,2004,107673.0,70460.0,29.827000,-95.660004,TX
2004-01-01,77449,Katy,Harris,0.422115,16508.811448,134945.348837,0.122337,123900.0,2004,128294.0,82716.0,29.837999,-95.734001,TX
2004-01-01,78660,Pflugerville,Travis,0.578674,23031.089035,180093.023256,0.127884,153800.0,2004,91300.0,99733.0,30.440001,-97.595001,TX
2004-01-01,77573,League City,Galveston,0.568086,23867.673325,199441.279070,0.119673,166700.0,2004,88131.0,121943.0,29.504000,-95.086998,TX
2004-01-01,79912,El Paso,El Paso,0.476452,13149.347629,171614.534884,0.076621,127400.0,2004,78267.0,77146.0,31.849001,-106.533997,TX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-04-01,76064,Maypearl,Ellis,0.659048,15308.985812,136593.604651,0.112077,174200.0,2018,1818.0,69350.0,32.301998,-97.037003,TX
2018-04-01,77577,Liverpool,Brazoria,0.761511,17597.138719,108870.348837,0.161634,149200.0,2018,1481.0,76382.0,29.284000,-95.281998,TX
2018-04-01,77663,Kountze,Hardin,0.312721,11184.253614,131325.581395,0.085164,148600.0,2018,NaN,NaN,NaN,NaN,NaN


**Create Affordability Score**

In [22]:
df_merged['median_house_price'] = df_merged.groupby('date').aggregate({'value':'median'})
 

In [23]:
df_merged['affordability'] = df_merged['value'] -df_merged['median_family_income']/.12

In [24]:
def normalize_data(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))
df_merged['affordability_score']  = normalize_data(df_merged['affordability'])
df_merged

,zipcode,City,CountyName,ROI,std,mean,CV,value,year,population,median_family_income,Latitude,Longitude,state,median_house_price,affordability,affordability_score
date,,,,,,,,,,,,,,,,,
2004-01-01,77084,Houston,Harris,0.376047,14170.210695,130143.604651,0.108881,119400.0,2004,107673.0,70460.0,29.827000,-95.660004,TX,121300.0,-467766.666667,0.604688
2004-01-01,77449,Katy,Harris,0.422115,16508.811448,134945.348837,0.122337,123900.0,2004,128294.0,82716.0,29.837999,-95.734001,TX,121300.0,-565400.000000,0.498403
2004-01-01,78660,Pflugerville,Travis,0.578674,23031.089035,180093.023256,0.127884,153800.0,2004,91300.0,99733.0,30.440001,-97.595001,TX,121300.0,-677308.333333,0.376578
2004-01-01,77573,League City,Galveston,0.568086,23867.673325,199441.279070,0.119673,166700.0,2004,88131.0,121943.0,29.504000,-95.086998,TX,121300.0,-849491.666667,0.189137
2004-01-01,79912,El Paso,El Paso,0.476452,13149.347629,171614.534884,0.076621,127400.0,2004,78267.0,77146.0,31.849001,-106.533997,TX,121300.0,-515483.333333,0.552743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-04-01,76064,Maypearl,Ellis,0.659048,15308.985812,136593.604651,0.112077,174200.0,2018,1818.0,69350.0,32.301998,-97.037003,TX,131600.0,-403716.666667,0.674414
2018-04-01,77577,Liverpool,Brazoria,0.761511,17597.138719,108870.348837,0.161634,149200.0,2018,1481.0,76382.0,29.284000,-95.281998,TX,131600.0,-487316.666667,0.583406
2018-04-01,77663,Kountze,Hardin,0.312721,11184.253614,131325.581395,0.085164,148600.0,2018,NaN,NaN,NaN,NaN,NaN,131600.0,NaN,NaN


In [29]:
top_20 = df_merged['affordability_score'].quantile(.2)
df_merged_20 = df_merged[(df_merged['affordability_score'] <top_20) 
                      & (df_merged['year']==2018) ]
df_merged_20

,zipcode,City,CountyName,ROI,std,mean,CV,value,year,population,median_family_income,Latitude,Longitude,state,median_house_price,affordability,affordability_score
date,,,,,,,,,,,,,,,,,
2018-01-01,77706,Beaumont,Jefferson,0.512958,10658.950187,140081.395349,0.076091,164600.0,2018,29474.0,87204.0,30.101999,-94.172997,TX,131100.0,-562100.000000,0.501996
2018-01-01,77627,Nederland,Jefferson,0.629268,11386.812213,109290.116279,0.104189,130200.0,2018,21698.0,95841.0,29.987000,-94.009003,TX,131100.0,-668475.000000,0.386195
2018-01-01,77657,Lumberton,Hardin,0.352490,12015.775520,149905.232558,0.080156,177000.0,2018,21345.0,91004.0,30.228001,-94.196999,TX,131100.0,-581366.666667,0.481022
2018-01-01,79015,Canyon,Randall,0.684360,15630.153550,142334.883721,0.109813,175900.0,2018,21925.0,94563.0,34.937000,-101.916000,TX,131100.0,-612125.000000,0.447538
2018-01-01,76310,Wichita Falls,Wichita,0.555917,9654.911517,121783.139535,0.079280,140400.0,2018,19187.0,84568.0,33.799000,-98.510002,TX,131100.0,-564333.333333,0.499565
2018-01-01,75707,Tyler,Smith,0.419763,10659.980756,152575.581395,0.069867,176000.0,2018,14232.0,90446.0,32.301998,-95.178001,TX,131100.0,-577716.666667,0.484995
2018-01-01,77563,Hitchcock,Galveston,0.492163,12820.083502,113886.046512,0.112569,145700.0,2018,11027.0,85040.0,29.304001,-95.031998,TX,131100.0,-562966.666667,0.501052
2018-01-01,77611,Bridge City,Orange,0.598824,12253.712804,108120.930233,0.113333,133700.0,2018,8578.0,89728.0,30.014000,-93.828003,TX,131100.0,-614033.333333,0.445460
2018-01-01,77625,Kountze,Hardin,0.325364,8796.072213,102522.674419,0.085796,125800.0,2018,9761.0,86045.0,30.377001,-94.374001,TX,131100.0,-591241.666667,0.470272


In [30]:
#Descriptive statistics of coefficients of variance.
print(df_merged_20.CV.describe())

#Define upper limit of CV according to risk profile.
upper_cv = df_merged_20.CV.quantile(.6)
print(f'\nCV upper limit: {upper_cv}')

#Get the 5 zipcodes with highest ROIs within the firms risk profile.
zc_best5 = df_merged[df_merged['CV']<upper_cv].sort_values('ROI',axis=0,ascending=False)[:5]
print('\n Best 5 Zipcodes:')
zc_best5[['zipcode','ROI','CV']]

count    60.000000
mean      0.111782
std       0.025736
min       0.069867
25%       0.095227
50%       0.111077
75%       0.126073
max       0.171584
Name: CV, dtype: float64

CV upper limit: 0.112874998405634

 Best 5 Zipcodes:


,zipcode,ROI,CV
date,,,
2013-10-01,78013,0.772834,0.11029
2010-01-01,78013,0.772834,0.11029
2009-12-01,78013,0.772834,0.11029
2012-03-01,78013,0.772834,0.11029
2013-08-01,78013,0.772834,0.11029


# EDA

Create DataFrame to perform stationarity checks and transformations

In [27]:
df_prices = pd.DataFrame(df['value'], index = df.index)
df_prices

,value
date,
2004-01-01,187700.0
2004-01-01,231800.0
2004-01-01,84000.0
2004-01-01,119400.0
2004-01-01,123900.0
...,...
2018-04-01,121600.0
2018-04-01,63300.0
2018-04-01,93500.0


**Mean prices by year**

In [ ]:
avg_prices = df_prices.groupby('date').aggregate({'value':'mean'})
avg_prices.plot()

In [ ]:
std_prices = df_prices.groupby('date').aggregate({'value':'std'})
std_prices.plot()

In [ ]:
med_prices = df_prices.groupby('date').aggregate({'value':'median'})
med_prices.plot()

In [ ]:
def stationarity_check(df):
    roll_mean = df.rolling(window=12, center=False).mean()
    roll_std = df.rolling(window=12, center=False).std()
    
    df_test = adfuller(df)
    print('Results of Dickey-Fuller Test: \n')

    dfoutput = pd.Series(df_test[0:4], index=['Test Statistic', 'p-value', 
                                             '#Lags Used', 'Number of Observations Used'])
    for key,value in df_test[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
    roll_mean.plot()
    roll_std.plot()
    return None

In [ ]:
results = stationarity_check(df_prices)
results

**Log Transformations**

In [ ]:
df_log = np.log(df_prices)
fig = plt.figure(figsize=(15,8))
plt.plot(df_log, color='blue');


**Weighted Rolling Mean**

In [ ]:
weight_roll_mean = df_log.ewm(halflife=4).mean()
df_log_wrm = df_log - weight_roll_mean
df_log_wrm.plot(figsize=(15,8))



In [ ]:
type(df_log_wrm.index)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

mod = ARIMA(df_log_wrm, order=(1, 0, 0))
res = mod.fit()
print(res.summary())